In [53]:
import os
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras


In [18]:
# constents
# workspace = "./autodl-tmp"
workspace = r".\dataset"
workspace


'.\\dataset'

In [50]:
def extract_bbox(silhouette):
    """
    Extract (x_min, y_min, x_max, y_max) according to the shape of silhouette
    Reference: https://stackoverflow.com/a/31402351/1471046
    """

    def convert_to_mask(silhouette, axis):
        return tf.math.count_nonzero(
            silhouette, axis=axis, keepdims=None, dtype=tf.bool
        )

    rows = convert_to_mask(silhouette, axis=0)
    print(rows)
    cols = convert_to_mask(silhouette, axis=1)
    print(cols)

    def coorindate(bin_mask) -> tf.Tensor:
        return tf.where(bin_mask)[:, -1]

    x_axis = coorindate(rows)
    x_min = x_axis[0]
    x_max = x_axis[-1]
    y_axis = coorindate(cols)
    y_min = y_axis[0]
    y_max = y_axis[-1]
    bbox = tf.concat([x_min, y_min, x_max, y_max], axis=0)
    print(bbox)
    return bbox


In [74]:
def draw_bbox(images, boxes):
    tableau_light = [
        (158, 218, 229),
        (219, 219, 141),
        (199, 199, 199),
        (247, 182, 210),
        (196, 156, 148),
        (197, 176, 213),
        (255, 152, 150),
        (152, 223, 138),
        (255, 187, 120),
        (174, 199, 232),
    ]
    tf.image.draw_bounding_boxes(
        images, boxes, tableau_light[0 : len(boxes)], name="Text"
    )


In [82]:
# test
def test_bbox():
    image = tf.expand_dims(
        tf.constant(
            [[0, 0, 0, 0], [0, 255, 255, 0], [0, 255, 255, 0], [0, 0, 0, 0]],
            dtype=tf.uint8,
        ),
        -1,
    )
    box = extract_bbox(image)
    print(box)


def test_bbox2():
    file_to_test = os.path.join(
        workspace, r"ground_truth", r"ISIC_0000000_segmentation.png"
    )
    file_content = tf.io.read_file(file_to_test)
    print(file_content.dtype)
    image = tf.io.decode_png(file_content)
    print(image[30])
    print(image.shape)
    box = extract_bbox(image)
    print(box)
    image_nhwc = tf.expand_dims(tf.expand_dims(image, 0), -1)
    print(image_nhwc.shape)
    draw_bbox(image_nhwc, [box])


test_bbox()
test_bbox2()


tf.Tensor(
[[False]
 [ True]
 [ True]
 [False]], shape=(4, 1), dtype=bool)
tf.Tensor(
[[False]
 [ True]
 [ True]
 [False]], shape=(4, 1), dtype=bool)
tf.Tensor([0 0 0 0], shape=(4,), dtype=int64)
tf.Tensor([0 0 0 0], shape=(4,), dtype=int64)
<dtype: 'string'>
tf.Tensor(
[[  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 

InvalidArgumentError: The rank of the images should be 4 [Op:DrawBoundingBoxesV2]

In [ ]:
def extract_annotations():
    directory = os.path.join(workspace, "annotation")
    os.makedirs(directory, exit_ok=True)

    for filename in os.listdir():
        if filename.endswith(".png"):
            file_id = filename.split(".")[0]
            bbox = extract_bbox(filename)
            content = f"""
"annotations": [
    {{
        "image_id": {file_id},
        "bbox": {bbox},
    }}
"""
extract_annotations()

In [20]:
SEED = 1
IMAGE_SIZE=(512,384)

In [ ]:
def transform(tensor,triple_channel=false):
  
  paddings = tf.constant([[1, 1, 1], [2, 2]]) if triple_channel else tf.constant()
  paddings=tf.constant([[channel,channel],[2,2]])
  tf.pad(
    tensor, paddings, mode='CONSTANT', constant_values=0, name=None)

In [ ]:
def augmentation():
    data_augmentation = keras.Sequential(
        [
            layers.Normalization(),
            layers.Resizing(image_size, image_size),
            layers.RandomFlip("horizontal"),
            layers.RandomRotation(factor=0.02),
            layers.RandomZoom(
                height_factor=0.2, width_factor=0.2
            ),
        ],
        name="data_augmentation",
    )
    return data_augmentation

In [ ]:
tf.keras.preprocessing.image_dataset_from_directory('./datasets/input')